In [ ]:
import numpy as np
import pandas as pd

# Apriorní pravděpodobnosti = pravděpodobnosti, které vyjadřují náš odhad před provedením pozorováním.
# Posteriózní pravděpodobnosti = pravděpodobnosti, které vyjadřují náš odhad po provedení pozorování.

# Hypotézy (stavy ženy).
hypotheses = ['Zcela se nelíbí', 'Nelíbí se', 'Neutrální', 'Líbí se', 'Zamilovanost']

# Apriorní pravděpodobnosti hypotéz P(H)
P_H = np.array([0.1, 0.15, 0.5, 0.2, 0.05])

# Pozorování (chování ženy).
observations = ["Kontakt pohledem", "Úsměv", "Požádala o pití", "Dívá se na jiné muže", "Krátké fráze", "Toaleta + make-up"]

# Pravděpodobnostní pozorování při daném stavu ženy.
P_E_given_H = np.array([
    [0.1, 0.2, 0.4, 0.2, 0.1],
    [0.1, 0.1, 0.2, 0.4, 0.2],
    [0.52, 0.45, 0.029, 0.0009, 0.0001],
    [0.3, 0.4, 0.2, 0.08, 0.02],
    [0.1, 0.4, 0.3, 0.1, 0.1],
    [0.05, 0.05, 0.1, 0.2, 0.6]
])
# display(pd.DataFrame(P_E_given_H, columns=hypotheses, index=observations))

# Posteriózní pravděpodobnosti.
posterior_probabilities = pd.DataFrame(index=hypotheses)
for i, observation in enumerate(observations):
    numerator = P_H * P_E_given_H[i] # Čitatel: P(Hi) * P(E|Hi) (tedy Součinový člen)
    print(numerator)
    denominator = np.round(np.sum(numerator), 3) # Jmenovatel: Σ P(Hi) * P(E|Hi) (tedy Normalizační člen)
    print(denominator)
    posterior_probability = np.round(numerator / denominator, 3) # P(Hi) * P(E|Hi) / Σ P(Hi) * P(E|Hi)
    print(f"{posterior_probability}\n")
    posterior_probabilities[observation] = posterior_probability
print(f"{posterior_probabilities.T}\n")

# Závěry (nejpravděpodobnější stav pro každé pozorování).
posterior_probabilities = posterior_probabilities.T
max_indices = posterior_probabilities.idxmax(axis=1)
max_values = posterior_probabilities.max(axis=1)
summary_table = pd.DataFrame({
    "Nejpravděpodobnější stav": max_indices,
    "Posteriorní pravděpodobnost": (max_values * 100).round(1).astype(str) + " %"
})
summary_table.index.name = "Pozorování"
header = f"{'Pozorování':30}{'Nejpravděpodobnější stav':30}{'Posteriorní pravděpodobnost'}"
print(header)
print("-" * len(header))
for idx, row in summary_table.iterrows():
    print(f"{idx:30}{row['Nejpravděpodobnější stav']:30}{row['Posteriorní pravděpodobnost']}")

[0.01  0.03  0.2   0.04  0.005]
0.285
[0.035 0.105 0.702 0.14  0.018]

[0.01  0.015 0.1   0.08  0.01 ]
0.215
[0.047 0.07  0.465 0.372 0.047]

[5.20e-02 6.75e-02 1.45e-02 1.80e-04 5.00e-06]
0.134
[0.388 0.504 0.108 0.001 0.   ]

[0.03  0.06  0.1   0.016 0.001]
0.207
[0.145 0.29  0.483 0.077 0.005]

[0.01  0.06  0.15  0.02  0.005]
0.245
[0.041 0.245 0.612 0.082 0.02 ]

[0.005  0.0075 0.05   0.04   0.03  ]
0.132
[0.038 0.057 0.379 0.303 0.227]

                      Zcela se nelíbí  Nelíbí se  Neutrální  Líbí se  \
Kontakt pohledem                0.035      0.105      0.702    0.140   
Úsměv                           0.047      0.070      0.465    0.372   
Požádala o pití                 0.388      0.504      0.108    0.001   
Dívá se na jiné muže            0.145      0.290      0.483    0.077   
Krátké fráze                    0.041      0.245      0.612    0.082   
Toaleta + make-up               0.038      0.057      0.379    0.303   

                      Zamilovanost  
Kontakt pohl

In [28]:
import numpy as np
import pandas as pd

# Monty Hall Problem
# ==================
# 1. Moderátor náhodně skryje cenu za jedny ze tří dveří.
# 2. Soutěžící si zvolí jedny dveře.
# 3. Moderátor otevře jedny z jiných dveří, za kterými není cena.
# 4. Soutěžící má možnost zůstat u svých původních dveří nebo změnit na zbývající zavřené dveře.

# Guest	= dveře zvolené soutěžícím.
# Prize	= dveře, za kterými je cena.
# Host	= dveře otevřené moderátorem (nikdy Prize, nikdy Guest).

# Možné dveře
doors = [0, 1, 2]

# Apriorní pravděpodobnosti (původní pravděpodobnost ceny za každými dveřmi)
P_H = np.array([1/3, 1/3, 1/3])

# Pravděpodobnosti chování moderátora
host_behaviour = {
    "Guest": [0, 0, 0, 1, 1, 1, 2, 2, 2],
    "Prize": [0, 1, 2, 0, 1, 2, 0, 1, 2],
    "Host(0)": [0,   0,   0,   0,   0.5, 1.0, 0,   1.0, 0.5],
    "Host(1)": [0.5, 0,   1.0, 0,   0,   0,   1.0, 0,   0.5],
    "Host(2)": [0.5, 1.0, 0,   1.0, 0.5, 0,   0,   0,   0]
}
df = pd.DataFrame(host_behaviour)

# Pro všechny možné volby soutěžícího a dveří, které může moderátor otevřít.
results = []
for guest in doors:
    for host in doors:
        if host == guest:
            continue  # Moderátor nikdy neotevírá dveře zvolené soutěžícím

        valid_prizes = [] # Možné dveře, za kterými může být výhra (v jedné iteraci).
        numerators = []   # Čitatel Bayesovy věty: P(H) * P(E | H) pro každou hypotézu Prize

        # Pro všechny možné umístění ceny
        for prize in doors:
            if prize == host:
                continue  # Moderátor nikdy neotevře dveře, za kterými je výhra

            # P(Host = host | Guest = guest, Prize = prize)
            p_e_given_h = df[
                (df["Guest"] == guest) &
                (df["Prize"] == prize)
            ][f"Host({host})"].values[0]

            # P(Prize = prize) * P(Host = host | Guest, Prize)
            valid_prizes.append(prize)
            numerators.append(P_H[prize] * p_e_given_h)

        # Normalizační člen ve jmenovateli
        numerators = np.array(numerators)
        normalization = numerators.sum()

        # Posteriózní pravděpodobnosti P(Prize = i | Guest, Host)
        posterior = numerators / normalization

        # Uložení výsledku pro každou možnou hypotézu o umístění výhry
        for prize, probability in zip(valid_prizes, np.round(posterior, 3)):
            results.append({
                "Guest": guest,             # Dveře zvolené soutěžícím
                "Host": host,               # Dveře otevřené moderátorem
                "Prize": prize,             # Dveře, za kterými je umístěna cena (hypoteticky)
                "Posterior": probability    # Výsledná posteriorní pravděpodobnost
            })
posterior_probabilities = pd.DataFrame(results)
posterior_probabilities = posterior_probabilities.sort_values(by=["Guest", "Host", "Prize"])
print("\nPosterior Probabilities:")
print(posterior_probabilities)

summary_rows = []
for (guest, host), group in posterior_probabilities.groupby(["Guest", "Host"]):
    p_keep = group[group["Prize"] == guest]["Posterior"].values[0]
    p_switch = group[group["Prize"] != guest]["Posterior"].sum()
    summary_rows.append({
        "Guest": guest,
        "Host": host,
        "P(Win|Keep)": round(p_keep, 3),
        "P(Win|Switch)": round(p_switch, 3),
        "Switch Recommended": p_switch > p_keep
    })

summary_df = pd.DataFrame(summary_rows)
print("\nBayesian Inference:")
print(summary_df)


Posterior Probabilities:
    Guest  Host  Prize  Posterior
0       0     1      0      0.333
1       0     1      2      0.667
2       0     2      0      0.333
3       0     2      1      0.667
4       1     0      1      0.333
5       1     0      2      0.667
6       1     2      0      0.667
7       1     2      1      0.333
8       2     0      1      0.667
9       2     0      2      0.333
10      2     1      0      0.667
11      2     1      2      0.333

Bayesian Inference:
   Guest  Host  P(Win|Keep)  P(Win|Switch)  Switch Recommended
0      0     1        0.333          0.667                True
1      0     2        0.333          0.667                True
2      1     0        0.333          0.667                True
3      1     2        0.333          0.667                True
4      2     0        0.333          0.667                True
5      2     1        0.333          0.667                True
